# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [65]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from shapely.geometry import Point

# Import API key
from api_keys import geoapify_key

In [66]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("city_weather_data.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,parksville,49.3163,-124.3195,10.80,99,100,11.32,CA,2024-11-11 02:55:34
1,1,utrik,11.2278,169.8474,28.53,76,3,8.76,MH,2024-11-11 02:55:35
2,2,kerikeri,-35.2268,173.9474,15.42,98,100,1.59,NZ,2024-11-11 02:54:14
3,3,puerto ayora,-0.7393,-90.3518,21.42,87,88,2.24,EC,2024-11-11 02:43:37
4,4,badger,64.8000,-147.5333,-17.05,84,0,2.06,US,2024-11-11 02:55:12


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [67]:
# Configure the map plot
city_map = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    tiles='OSM', 
    hover_cols=['City', 'Country', 'Max Temp', 'Humidity'],
    size='Humidity',  # Set the size of the points based on humidity
    size_max=20,      # Adjust the maximum size of the points
    color='Humidity', # Optional: color points based on humidity as well
    cmap='Category10'      # Optional: choose a color map for humidity
)


In [68]:
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [69]:
# Define your criteria
max_temp_min = 20  # Minimum Max Temp
max_temp_max = 30  # Maximum Max Temp
min_humidity = 50  # Minimum Humidity

# Narrow down the DataFrame based on criteria
filtered_cities = city_data_df[
    (city_data_df['Max Temp'] >= max_temp_min) & 
    (city_data_df['Max Temp'] <= max_temp_max) & 
    (city_data_df['Humidity'] >= min_humidity)
]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

In [70]:
# Display the filtered DataFrame
filtered_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,utrik,11.2278,169.8474,28.53,76,3,8.76,MH,2024-11-11 02:55:35
3,3,puerto ayora,-0.7393,-90.3518,21.42,87,88,2.24,EC,2024-11-11 02:43:37
7,7,puerto ayora,-0.7393,-90.3518,21.42,87,88,2.24,EC,2024-11-11 02:43:37
8,8,pitimbu,-7.4706,-34.8086,25.09,73,100,5.14,BR,2024-11-11 02:55:43
10,10,freetown,8.4840,-13.2299,26.43,79,3,1.11,SL,2024-11-11 02:54:52
...,...,...,...,...,...,...,...,...,...,...
1417,1417,west island,-12.1568,96.8225,28.99,74,20,7.72,CC,2024-11-11 03:23:03
1422,1422,adamstown,-25.0660,-130.1015,21.77,76,100,3.80,PN,2024-11-11 03:19:18
1428,1428,hilo,19.7297,-155.0900,24.85,87,40,1.54,US,2024-11-11 03:18:50
1432,1432,adamstown,-25.0660,-130.1015,21.77,76,100,3.80,PN,2024-11-11 03:19:18


### Step 3: Create a new DataFrame called `hotel_df`.

In [71]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()  


In [72]:
# Add an empty column for hotel names
hotel_df['Hotel Name'] = None
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,utrik,MH,11.2278,169.8474,76,None
3,puerto ayora,EC,-0.7393,-90.3518,87,None
7,puerto ayora,EC,-0.7393,-90.3518,87,None
8,pitimbu,BR,-7.4706,-34.8086,73,None
10,freetown,SL,8.4840,-13.2299,79,None
...,...,...,...,...,...,...
1417,west island,CC,-12.1568,96.8225,74,None
1422,adamstown,PN,-25.0660,-130.1015,76,None
1428,hilo,US,19.7297,-155.0900,87,None
1432,adamstown,PN,-25.0660,-130.1015,76,None


In [73]:
# Set parameters to search for a hotel
radius = 10000  # Set the search radius to 10,000 meters
params = {
    "apiKey": "034181f44e6a466d9f0aaa65e6fc2b92",  
    "radius": radius,
    "limit": 1,  # Limit the results to 1 hotel
    "categories": "accommodation.hotel"  # Use the correct category for hotels
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"] 
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()
    
    # Print the API response for debugging
    print(name_address)

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try: 
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'type': 'FeatureCollection', 'features': []}
utrik - nearest hotel: No hotel found
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Hostal La Mirada De Solitario George', 'country': 'Ecuador', 'country_code': 'ec', 'state': 'Galápagos', 'county': 'Cantón Santa Cruz', 'city': 'Puerto Ayora', 'municipality': 'Parroquia Bella Vista', 'postcode': '200102', 'district': 'Pampas Coloradas', 'street': 'Petrel', 'lon': -90.3178804, 'lat': -0.7396457, 'state_code': 'W', 'formatted': 'Hostal Mirada Del Solitario George Hostel, Petrel, 200102, Puerto Ayora, Ecuador', 'address_line1': 'Hostal Mirada Del Solitario George Hostel', 'address_line2': 'Petrel, 200102, Puerto Ayora, Ecuador', 'categories': ['accommodation', 'accommodation.hotel', 'internet_access', 'internet_access.free'], 'details': ['details', 'details.facilities'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Da

,City,Country,Lat,Lng,Humidity,Hotel Name
1,utrik,MH,11.2278,169.8474,76,No hotel found
3,puerto ayora,EC,-0.7393,-90.3518,87,Hostal La Mirada De Solitario George
7,puerto ayora,EC,-0.7393,-90.3518,87,Hostal La Mirada De Solitario George
8,pitimbu,BR,-7.4706,-34.8086,73,Pousada Brisa Mar
10,freetown,SL,8.4840,-13.2299,79,Formal Leona hotel
...,...,...,...,...,...,...
1417,west island,CC,-12.1568,96.8225,74,Cocos Village Bungalows
1422,adamstown,PN,-25.0660,-130.1015,76,No hotel found
1428,hilo,US,19.7297,-155.0900,87,Dolphin Bay Hotel
1432,adamstown,PN,-25.0660,-130.1015,76,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [75]:
# Create a map using hvplot
map_plot = hotel_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    tiles='OSM', 
    hover_cols=['Hotel Name','City', 'Country'],
    size='Humidity',  # Set the size of the points based on humidity
    size_max=20,      # Adjust the maximum size of the points
    color='Humidity', # Optional: color points based on humidity as well
    cmap='Category10',      # Optional: choose a color map for humidity
    text='Hotel Name'  # Display hotel names next to the points

)

#display map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,City,Country)